In [1]:
!pip install tensorflow_datasets


     |████████████████████████████████| 3.4MB 546kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for promise: filename=promise-2.3-cp36-none-any.whl size=21494 sha256=1181f69929f473eb9116a6a61592ff44effe9939ef0378eec2333c1c4cd9742d
  Stored in directory: /root/.cache/pip/wheels/19/49/34/c3c1e78bcb954c49e5ec0d31784fe63d14d427f316b12fbde9
Successfully built promise


In [2]:

import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [3]:
mbti_dataset_line = tf.data.TextLineDataset("../input/mbti-clean-with-categories/mbti_clean.csv")

In [4]:
for ex in mbti_dataset_line.take(5):
  print(ex)

tf.Tensor(b'type,posts,typescat,IE,NS,TF,JP', shape=(), dtype=string)
tf.Tensor(b'INFJ,hello  ptypetoken  eostokendot  sorri  hear  your distress eostokendot   onli natur   relationship    perfect   time  everi moment  exist eostokendot    figur  hard time  time  growth  eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ,prozac wellbrutin  least thirti minut  move your leg   dont mean move them while sit  your same desk chair weed  moder mayb  edibl   healthier altern eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ,basic come  with three item youv determin that each type  whichev type  want   would more than like  given each type cognit function  whatnot when left  eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ, thing  moder eostokendot   sim  inde  video game   good   that eostokendot  note  good   that  somewhat subject  that    complet promot 

In [5]:
def label(line):
  label =  tf.strings.substr([line],[-10],[1])
  if label[0]==',':
    label = tf.strings.substr([line],[-9],[1])
  else:
    label = tf.strings.substr([line],[-10],[2])
  labelnum=tf.strings.to_number(label,tf.int64)
  line= tf.strings.substr([line],[6],(tf.strings.length([line])-17))
  return line[0], labelnum[0]


In [6]:

mbti_dataset_line = mbti_dataset_line.skip(1).map(lambda line: label(line))


In [7]:
for ex in mbti_dataset_line.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'ello  ptypetoken  eostokendot  sorri  hear  your distress eostokendot   onli natur   relationship    perfect   time  everi moment  exist eostokendot    figur  hard time  time  growth  eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'rozac wellbrutin  least thirti minut  move your leg   dont mean move them while sit  your same desk chair weed  moder mayb  edibl   healthier altern eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'asic come  with three item youv determin that each type  whichev type  want   would more than like  given each type cognit function  whatnot when left  eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'thing  moder eostokendot   sim  inde  video game   good   that eo

In [8]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [9]:
mbti_dataset_line = mbti_dataset_line.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [10]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in mbti_dataset_line:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

83501

In [11]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [12]:
example_text = next(iter(mbti_dataset_line))[0].numpy()
print(example_text)

b' ptypetoken blog eostokendot   poster write that  have  abil   selfawar  introvert feel allow   know when were do someth wrong eostokendot     feel  becaus when   younger  eostokendot  eostokendot  eostokendot    '


In [13]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[39178, 1943, 38607, 40934, 69066, 31974, 22491, 71459, 51472, 55739, 40263, 563, 4141, 72317, 35320, 643, 44350, 40252, 38607, 40263, 55260, 72317, 44254, 38607, 38607, 38607]


In [14]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = mbti_dataset_line.map(encode_map_fn)

In [15]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [16]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(31,), dtype=int64, numpy=
 array([39178,  1943, 38607, 40934, 69066, 31974, 22491, 71459, 51472,
        55739, 40263,   563,  4141, 72317, 35320,   643, 44350, 40252,
        38607, 40263, 55260, 72317, 44254, 38607, 38607, 38607,     0,
            0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=9>)

In [17]:
vocab_size += 1

In [18]:
model = tf.keras.Sequential()

In [19]:
model.add(tf.keras.layers.Embedding(vocab_size+1, 64))

In [20]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [21]:
# One or more dense layers.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(16, activation='softmax'))

In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.fit(train_data, epochs=1, validation_data=test_data)

4203/4203 [==============================] - 372s 89ms/step - loss: 2.2337 - accuracy: 0.2253 - val_loss: 2.2152 - val_accuracy: 0.2294


In [24]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

     79/Unknown - 10s 132ms/step - loss: 2.2152 - accuracy: 0.2294
Eval loss: 2.215, Eval accuracy: 0.229
